# A Visual Proof: Difference of Squares

From time to time I'd read about geniuses for whom it wasn't sufficient just to be told by a teacher that something was so; they needed to prove it for themselves. Not I. If a teacher told me something, that was sufficient. Still, in my defense, sometimes a topic would trigger a "huh, I wonder why that is?" This is an example of one of those.

Though long lost in memory, I was reminded of that flicker of curiousity when I came across [this neat little post](https://www.futilitycloset.com/2024/12/15/tidy-2/). What follows is a `Jupyter Notebook` + animated `ipycanvas` implementation of that post.

#### Steps
1. [Imports](#imports)
1. [Constants](#constants)
1. [Draw functions for each step](#step-draw-functions)
1. [Animation updaters](#updaters)
1. [Creating the elements](#elements)
1. [Defining the layout and displaying it](#display)

#### Imports
<a id="imports"></a>Included in the list of imports are two custom modules, both of which can be found in the local `utils/` package. The first is `CanvasRenderer`. It provides convenience methods for updating the canvas. The second is the `RenderUtils` class. We use it in each of the step `draw()` functions. Unless you're interested, you don't need to worry about either of these. Their underlying and common purpose is to hide the nitty-gritties of drawing to the canvas.

In [1]:
from IPython.display import display
from ipycanvas import RoughCanvas, hold_canvas
from ipywidgets import HBox, HTML, IntSlider, jslink, Layout, Play, VBox
from utils.graphics.canvas_renderer import CanvasRenderer
from utils.notebooks.difference_of_squares import CANVAS_HEIGHT, CANVAS_WIDTH, Color, PLOT_CONTENT_B_DIMENSION, RenderUtils

#### Constants
<a id="constants"></a>We'll define some constants used in the animation. Additional constants and an enum are imported from the utility module, above.

In [2]:
ANIMATION_START_STEP = 1
ANIMATION_MAX_STEPS = 6
ANIMATION_INTERVAL = 2000 # The pause before progessing to the next frame

#### Draw functions for each step
<a id="step-draw-functions"></a>For each step of the animation, we need to draw the appropriate representation. Here's where we make use of the methods defined in `RenderUtils`.

In [3]:
def draw_step1():
    with hold_canvas():
        (RenderUtils
            .clear()
            .a_rect()
            .b_rect()
            .a_bar()
            .b_border()
            .a_plus_b_border()
            .label_a_top()
            .label_a_left()
            .label_b_box_right()
            .label_b_box_bottom()
            .b_box_snip())


def draw_step2():
    with hold_canvas():
        (RenderUtils
            .clear()
            .a_minus_b()
            .a_bar()
            .a_minus_b_line()
            .label_a_top()
            .label_a_left()
            .label_b_left()
            .label_b_right()
            .label_a_minus_b_right()
            .label_a_minus_b_bottom()
            .a_minus_b_snip())


def draw_step3():
    b_offset = 15
    with hold_canvas():
        (RenderUtils
            .clear()
            .a_minus_b_rect()
            .b_rect_horizontal(b_offset)
            .label_a_top()
            .label_b_left(b_offset)
            .label_b_right(b_offset)
            .label_a_minus_b_left()
            .label_a_minus_b_right()
            .label_a_minus_b_bottom(b_offset))


def draw_step4():
    b_offset = 15
    with hold_canvas():
        (RenderUtils
            .clear()
            .a_minus_b_rect()
            .b_rect_vertical(b_offset)
            .label_a_top()
            .label_a_bottom()
            .label_b_top(b_offset)
            .label_b_bottom(b_offset)
            .label_a_minus_b_left()
            .label_a_minus_b_right(PLOT_CONTENT_B_DIMENSION + b_offset))


def draw_step5():
    with hold_canvas():
        (RenderUtils
            .clear()
            .a_plus_b_rect()
            .a_plus_b_line()
            .a_plus_b_paste()
            .label_a_top()
            .label_a_bottom()
            .label_b_top()
            .label_b_bottom()
            .label_a_minus_b_left()
            .label_a_minus_b_right(PLOT_CONTENT_B_DIMENSION))


def draw_step6():
    with hold_canvas():
        (RenderUtils
            .clear()
            .a_plus_b_rect()
            .label_a_minus_b_left()
            .label_a_minus_b_right(PLOT_CONTENT_B_DIMENSION)
            .label_a_plus_b_top()
            .label_a_plus_b_bottom())


#### Animation updaters
<a id="updaters"></a>There two elements we want to update at each animation step. The most obvious of course is the canvas. The other is the title.

[`Jupyter Widgets`](https://ipywidgets.readthedocs.io/en/stable/) commonly make use of the [Observer Pattern](https://en.wikipedia.org/wiki/Observer_pattern). In our case, when the slider changes its value (or has its value changed), the associated observer will invoke our `update_step()` function. That plucks the current step value and sends it along to our display function, `update_display()`.

In [4]:
def update_step(update):
    def fn(change):
        value = change['new']
        update(value)
    return fn

def update_title(title, step):
    left_side = 'a<sup>2</sup> - b<sup>2</sup></span>'
    right_side = '(a + b)(a - b)'

    # Highlight the relevant part of the equation.
    if step in (1, 2):
        left_side = f'<span style="font-weight: bold; color: {Color.BLUE.value}">{left_side}</span>'
    if step in (5, 6):
        right_side = f'<span style="font-weight: bold; color: {Color.BLUE.value}">{right_side}</span>'

    title.value = f'''<div style="margin-bottom: 12px;">
        <h2 style="font-weight: normal; color: {Color.GRAY.value}">{left_side} = {right_side}</h2>
        </div>'''

def update_canvas(step):
    match step:
        case 2:
            draw_step2()
        case 3:
            draw_step3()
        case 4:
            draw_step4()
        case 5:
            draw_step5()
        case 6:
            draw_step6()
        case _:
            draw_step1()


def update_display(title):
    def fn(step):
        CanvasRenderer.restore(CanvasRenderer.defaults())
        update_title(title, step)
        update_canvas(step)
    return fn


#### Creating the elements
<a id="elements"></a>Our presentation is comprised of 4 elements: the title, the canvas, the slider control, and the animation control. The title, slider, and animation control come from [`Jupyter Widgets`](https://ipywidgets.readthedocs.io/en/stable/). The canvas comes from [`ipycanvas`](https://ipycanvas.readthedocs.io/en/latest/).

The animation control, as it plays, updates the value of the slider. The slider&mdash;in turn and as noted above&mdash;observes updates, kicking off the sequence for the current step value.

In [5]:
title = HTML(
    value='<h2 style="font-weight: normal;"><span style="font-weight: bold">a<sup>2</sup> - b<sup>2</sup></span> <span style="color: #555">= (a + b)(a - b)</span></h2>',
)

canvas = RoughCanvas(width=CANVAS_WIDTH, height=CANVAS_HEIGHT)
CanvasRenderer.set_canvas(canvas)
CanvasRenderer.set_defaults({
    'font': canvas.font,
    'fill_style': canvas.fill_style,
    'stroke_style': canvas.stroke_style,
    'line_dash': canvas.get_line_dash(),
})
RenderUtils.set_renderer(CanvasRenderer)

slider = IntSlider(
    description="Step",
    value=ANIMATION_START_STEP,
    min=1,
    max=ANIMATION_MAX_STEPS
)
update = update_step(update_display(title))
slider.observe(update, names="value")

play = Play(
    interval=ANIMATION_INTERVAL,
    value=ANIMATION_START_STEP,
    min=1,
    max=ANIMATION_MAX_STEPS,
    step=1,
    description="Step",
    disabled=False,
    repeat=True
)
jslink((play, 'value'), (slider, 'value')); # The ; semicolon suppresses a print out of the type of object.

#### Defining the layout and displaying it
<a id="display"></a>Now that the individual elements have been defined and created, let's put them into a layout and display them!

In [6]:
update({'new': ANIMATION_START_STEP}) # Show step 1 by default
layout = VBox([
    HBox([title], layout=Layout(width='100%', justify_content='center')),
    HBox([play, slider], layout=Layout(width='100%', justify_content='center')),
    HBox([canvas], layout=Layout(width='100%', justify_content='center')),
])
display(layout)
